## <div align = center> Immersion à Data : *Rapport de synthèse*

In [231]:
# importations 

import pandas as pd
import numpy as np
import sklearn as sk
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt

In [232]:
df = pd.read_csv("datas.csv", sep = ";")
df.head()

,id,sex,dob,age,age_cat,social_cat,priors_count,c_jail_in,c_jail_out,c_case_number,...,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_offense_date,r_charge_desc,is_violent_recid,decile_score,score_text,screening_date
0,1,Male,1947-04-18,69,Greater than 45,Other,0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,...,Aggravated Assault w/Firearm,0,NaN,O,NaN,NaN,0,1,Low,2013-08-14
1,2,Male,1985-02-06,31,25 - 45,Worker,0,NaN,NaN,NaN,...,NaN,-1,NaN,O,NaN,NaN,0,5,Medium,2014-12-31
2,3,Male,1982-01-22,34,25 - 45,Farmer,0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,...,Felony Battery w/Prior Convict,1,13009779CF10A,F,2013-07-05,Felony Battery (Dom Strang),1,3,Low,2013-01-27
3,4,Male,1991-05-14,24,Less than 25,Farmer,4,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,...,Possession of Cocaine,1,13011511MM10A,M,2013-06-16,Driving Under The Influence,0,4,Low,2013-04-14
4,5,Male,1993-01-21,23,Less than 25,Farmer,1,NaN,NaN,13000570CF10A,...,Possession of Cannabis,0,NaN,O,NaN,NaN,0,8,High,2013-01-13


In [233]:
df.describe(include="all")

,id,sex,dob,age,age_cat,social_cat,priors_count,c_jail_in,c_jail_out,c_case_number,...,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_offense_date,r_charge_desc,is_violent_recid,decile_score,score_text,screening_date
count,11757.000000,11757,11757,11757.000000,11757,11757,11757.000000,10577,10577,11015,...,11008,11757.000000,3703,11757,3703,3643,11757.000000,11757.000000,11742,11757
unique,NaN,2,7800,NaN,3,6,NaN,10577,10517,11015,...,531,NaN,3703,3,1090,352,NaN,NaN,3,704
top,NaN,Male,1991-08-12,NaN,25 - 45,Farmer,NaN,2013-08-13 06:03:42,2014-02-12 10:41:00,13011352CF10A,...,arrest case no charge,NaN,13009779CF10A,O,2014-12-08,Driving License Suspended,NaN,NaN,Low,2013-03-20
freq,NaN,9336,6,NaN,6649,5813,NaN,1,4,1,...,1858,NaN,1,8056,12,279,NaN,NaN,6607,39
mean,5879.000000,NaN,NaN,35.143319,NaN,NaN,3.082164,NaN,NaN,NaN,...,NaN,0.253806,NaN,NaN,NaN,NaN,0.075019,4.371268,NaN,NaN
std,3394.097892,NaN,NaN,12.022894,NaN,NaN,4.687410,NaN,NaN,NaN,...,NaN,0.558324,NaN,NaN,NaN,NaN,0.263433,2.877598,NaN,NaN
min,1.000000,NaN,NaN,18.000000,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,-1.000000,NaN,NaN,NaN,NaN,0.000000,-1.000000,NaN,NaN
25%,2940.000000,NaN,NaN,25.000000,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,2.000000,NaN,NaN
50%,5879.000000,NaN,NaN,32.000000,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,4.000000,NaN,NaN
75%,8818.000000,NaN,NaN,43.000000,NaN,NaN,4.000000,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,0.000000,7.000000,NaN,NaN


In [234]:
# on transforme les dates en flotant

df['year_in'] = pd.DatetimeIndex(df['c_jail_in']).year
df['month_in'] = pd.DatetimeIndex(df['c_jail_in']).month
df['month_out'] = pd.DatetimeIndex(df['c_jail_out']).month
df['year_out'] = pd.DatetimeIndex(df['c_jail_out']).year

df['day_out'] = pd.DatetimeIndex(df['c_jail_out']).day
df['day_in'] = pd.DatetimeIndex(df['c_jail_in']).day

df['date_in'] = df['year_in'] + df['month_in']/12 + df['day_in']/365
df['date_out'] = df['year_out'] + df['month_out']/12 + df['day_out']/365
df['duree_prison'] = df['date_out'] - df['date_in']

In [235]:
df.head()

,id,sex,dob,age,age_cat,social_cat,priors_count,c_jail_in,c_jail_out,c_case_number,...,screening_date,year_in,month_in,month_out,year_out,day_out,day_in,date_in,date_out,duree_prison
0,1,Male,1947-04-18,69,Greater than 45,Other,0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,...,2013-08-14,2013.0,8.0,8.0,2013.0,14.0,13.0,2013.702283,2013.705023,0.002740
1,2,Male,1985-02-06,31,25 - 45,Worker,0,NaN,NaN,NaN,...,2014-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Male,1982-01-22,34,25 - 45,Farmer,0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,...,2013-01-27,2013.0,1.0,2.0,2013.0,5.0,26.0,2013.154566,2013.180365,0.025799
3,4,Male,1991-05-14,24,Less than 25,Farmer,4,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,...,2013-04-14,2013.0,4.0,4.0,2013.0,14.0,13.0,2013.368950,2013.371689,0.002740
4,5,Male,1993-01-21,23,Less than 25,Farmer,1,NaN,NaN,13000570CF10A,...,2013-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:
new_df = df[['id', 'age', 'priors_count', 'is_recid', 'duree_prison', 'sex', 'social_cat', 'decile_score', 'c_charge_degree', 'is_violent_recid']]
new_df = new_df.set_index('id')
new_df.describe()

,age,priors_count,is_recid,duree_prison,decile_score,is_violent_recid
count,11757.000000,11757.000000,11757.000000,10577.000000,11757.000000,11757.000000
mean,35.143319,3.082164,0.253806,0.069788,4.371268,0.075019
std,12.022894,4.687410,0.558324,0.201400,2.877598,0.263433
min,18.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000
25%,25.000000,0.000000,0.000000,0.002740,2.000000,0.000000
50%,32.000000,1.000000,0.000000,0.002740,4.000000,0.000000
75%,43.000000,4.000000,1.000000,0.035616,7.000000,0.000000
max,96.000000,43.000000,1.000000,5.897489,10.000000,1.000000


In [237]:
# On commence par construire la table de contingence entre les données qui nous intéressent
intparam1 = "sex" # Le paramètre d'intérêt
intparam2 = "social_cat"

# choix de la dataframe
df_cont = df#[df["sex"] == "Female"]

cont_table = df_cont[[intparam1, intparam2]].pivot_table(index = intparam1, columns=intparam2, aggfunc=len).fillna(0).astype(int)
cont_table

social_cat,CivilServant,Farmer,Manager,OfficeWorker,Other,Worker
sex,,,,,,
Female,10,1116,7,189,131,968
Male,30,4697,51,911,530,3117


In [238]:
st_chi2, st_p, st_dof, st_exp = chi2_contingency(cont_table)
st_p

5.6259402176780564e-08

In [239]:
df_cont["decile_score"].mean(), df["decile_score"].mean()

(4.371268180658332, 4.371268180658332)

In [248]:
recid_df = df[df["is_recid"] == 1]
recid_df["offense_date"] = pd.DatetimeIndex(recid_df["r_offense_date"]).year + pd.DatetimeIndex(recid_df["r_offense_date"]).month/12 + pd.DatetimeIndex(recid_df["r_offense_date"]).day/365
recid_df["time_before_offense"] = recid_df["offense_date"] - recid_df["date_out"]
recid_df["time_before_offense"].describe()

/var/folders/y2/wrgv4_bd7csfrv7bblb28_zc0000gn/T/ipykernel_10531/693948036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recid_df["offense_date"] = pd.DatetimeIndex(recid_df["r_offense_date"]).year + pd.DatetimeIndex(recid_df["r_offense_date"]).month/12 + pd.DatetimeIndex(recid_df["r_offense_date"]).day/365
/var/folders/y2/wrgv4_bd7csfrv7bblb28_zc0000gn/T/ipykernel_10531/693948036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recid_df["time_before_offense"] = recid_df["offense_date"] - recid_df[

count    3634.000000
mean        0.703574
std         0.706591
min        -2.269178
25%         0.172432
50%         0.535616
75%         1.099372
max         3.191324
Name: time_before_offense, dtype: float64

Ils récidivent assez vite 